# 태스크 2a: Titan Text Premier를 통한 작은 파일의 텍스트 요약


이 노트북에서는 작은 텍스트 문자열을 Amazon Bedrock API(Titan Text 모델 사용)에 직접 수집하고 입력 텍스트를 요약하도록 지시합니다. 입력 텍스트 길이가 모델의 컨텍스트 크기 한도 내에 있는 경우 이 접근 방식을 적용하여 통화 대본, 회의 대본, 도서, 기사, 블로그 게시물 및 기타 관련 콘텐츠를 요약할 수 있습니다.

## 태스크 2a.1: 환경 설정

이 태스크에서는 환경을 설정합니다.

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3
import botocore

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))



## 태스크 2a.2: 요약할 텍스트로 프롬프트 작성

이 태스크에서는 파운데이션 모델에서 지원하는 최대 길이보다 토큰이 적은 짧은 텍스트 구절을 사용합니다. 이 실습의 샘플 입력 텍스트로는 Amazon Bedrock을 발표하는 [AWS 블로그 게시물](https://aws.amazon.com/jp/blogs/machine-learning/announcing-new-tools-for-building-with-generative-ai-on-aws/)의 단락을 사용합니다.

이 프롬프트는 `Please provide a summary of the following text.` 지침으로 시작합니다. 

In [ ]:
prompt_data = """

Please provide a summary of the following text:

AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
for text and images—including Amazons Titan FMs, which consist of two new LLMs we’re also announcing \
today—through a scalable, reliable, and secure AWS managed service. With Bedrock’s serverless experience, \
customers can easily find the right model for what they’re trying to get done, get started quickly, privately \
customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
"""

## 태스크 2a.3: 프롬프트 및 추론 파라미터로 요청 본문 생성 

이 태스크에서는 위의 프롬프트와 추론 파라미터로 요청 본문을 생성합니다.

In [ ]:
# request body
body = json.dumps({
    "inputText": prompt_data, 
    "textGenerationConfig":{
        "maxTokenCount":2048,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
        }
    }) 

## 태스크 2a.4: Boto3를 통해 파운데이션 모델 호출

이 태스크에서는 요청 파라미터 `modelId`, `accept` 및 `contentType`을 지정하여 Amazon Bedrock으로 API 요청을 전송합니다. 제공된 프롬프트에 따라 Amazon Bedrock의 파운데이션 모델이 입력 텍스트를 요약합니다.

### 전체 출력 생성

기본적으로 Amazon Bedrock 서비스에서는 주어진 프롬프트에 대한 전체 요약을 단일 출력으로 생성합니다. 모델 출력에 토큰이 많으면 속도가 느려질 수 있습니다. 

In [ ]:
#model configuration and invoke the model
modelId = 'amazon.titan-text-premier-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"

try:

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    outputText = response_body.get('results')[0].get('outputText')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

print(outputText)

### 출력 생성 스트리밍

다음으로, 사용자가 출력을 생성하면서 사용할 수 있도록 Amazon Bedrock의 invoke_model_with_response_stream API를 사용하여 모델 출력을 스트리밍하는 방법을 살펴봅니다. 이 API에서는 전체 출력을 한 번에 생성하지 않고 모델이 생성될 때 더 작은 출력 청크를 전송하는 ResponseStream을 반환합니다. 이러한 스트리밍 출력을 연속적이고 사용 가능한 보기로 표시할 수 있습니다.

In [ ]:
#invoke model with response stream
modelId = 'amazon.titan-text-premier-v1:0'
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = list(stream)
output

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
modelId = 'amazon.titan-text-premier-v1:0'
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

지금까지 Amazon Bedrock API 액세스에 boto3 SDK를 사용하는 실험을 해보았습니다. 이 SDK에서는 Bedrock 기능에 대한 기본 프로그래밍 방식 액세스를 제공합니다. 이 API를 활용하여 두 가지 사용 사례를 구현할 수 있었습니다. 1) AWS 뉴스 콘텐츠의 전체 텍스트 요약을 한 번에 생성하고, 2) 증분 처리를 위해 요약 출력을 청크 단위로 스트리밍하는 것입니다.

### 직접 해보기
- 프롬프트를 특정 사용 사례로 변경하고 다양한 모델의 출력을 평가하십시오.
- 토큰 길이를 조작하면서 서비스의 지연 시간 및 응답성을 이해해 보십시오.
- 다양한 프롬프트 엔지니어링 원칙을 적용하여 더 나은 출력을 얻을 수 있습니다.

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 이 노트북 파일을 닫고 **Task2b_ko_kr.ipynb**를 계속합니다.